# Analysing Data

In [24]:
import pandas as pd

metad = pd.read_csv("data/HAM10000_metadata.csv")
metad

L8 = pd.read_csv("data/hmnist_8_8_L.csv")
RGB8 = pd.read_csv("data/hmnist_8_8_RGB.csv")
L28 = pd.read_csv("data/hmnist_28_28_L.csv")
RGB28 = pd.read_csv("data/hmnist_28_28_L.csv") 


metad.groupby("dx").count()
metad

,lesion_id,image_id,dx,dx_type,age,sex,localization
0,HAM_0000118,ISIC_0027419,bkl,histo,80.0,male,scalp
1,HAM_0000118,ISIC_0025030,bkl,histo,80.0,male,scalp
2,HAM_0002730,ISIC_0026769,bkl,histo,80.0,male,scalp
3,HAM_0002730,ISIC_0025661,bkl,histo,80.0,male,scalp
4,HAM_0001466,ISIC_0031633,bkl,histo,75.0,male,ear
...,...,...,...,...,...,...,...
10010,HAM_0002867,ISIC_0033084,akiec,histo,40.0,male,abdomen
10011,HAM_0002867,ISIC_0033550,akiec,histo,40.0,male,abdomen
10012,HAM_0002867,ISIC_0033536,akiec,histo,40.0,male,abdomen
10013,HAM_0000239,ISIC_0032854,akiec,histo,80.0,male,face


In [25]:
tags = metad.dx.unique().tolist()
tags

['bkl', 'nv', 'df', 'mel', 'vasc', 'bcc', 'akiec']

In [26]:
bkl_ids = []
nv_ids = []
df_ids = []
mel_ids = []
vasc_ids = []
bcc_ids = []
akiec_ids = []

super_list = [bkl_ids,
              nv_ids,
              df_ids,
              mel_ids,
              vasc_ids,
              bcc_ids,
              akiec_ids]


for i in range(len(tags)):
    super_list[i] = (metad[metad['dx'] == tags[i]]['image_id'].values.tolist())

In [27]:
super_list

[['ISIC_0027419',
  'ISIC_0025030',
  'ISIC_0026769',
  'ISIC_0025661',
  'ISIC_0031633',
  'ISIC_0027850',
  'ISIC_0029176',
  'ISIC_0029068',
  'ISIC_0025837',
  'ISIC_0025209',
  'ISIC_0025276',
  'ISIC_0029396',
  'ISIC_0025984',
  'ISIC_0025767',
  'ISIC_0032417',
  'ISIC_0031326',
  'ISIC_0025915',
  'ISIC_0031029',
  'ISIC_0029836',
  'ISIC_0032129',
  'ISIC_0032343',
  'ISIC_0025033',
  'ISIC_0027310',
  'ISIC_0032128',
  'ISIC_0025937',
  'ISIC_0027828',
  'ISIC_0029291',
  'ISIC_0030698',
  'ISIC_0025567',
  'ISIC_0031753',
  'ISIC_0026835',
  'ISIC_0031159',
  'ISIC_0031017',
  'ISIC_0024981',
  'ISIC_0027815',
  'ISIC_0024324',
  'ISIC_0029559',
  'ISIC_0030661',
  'ISIC_0027053',
  'ISIC_0028560',
  'ISIC_0031650',
  'ISIC_0029161',
  'ISIC_0026273',
  'ISIC_0025076',
  'ISIC_0029687',
  'ISIC_0025642',
  'ISIC_0025819',
  'ISIC_0032013',
  'ISIC_0031691',
  'ISIC_0025419',
  'ISIC_0030105',
  'ISIC_0025016',
  'ISIC_0029147',
  'ISIC_0029301',
  'ISIC_0026637',
  'ISIC_00

In [28]:
import numpy as np
from tensorflow.keras.applications.mobilenet_v2 import preprocess_input

def get_preprocessed_images(images_ids: list, image_size: tuple) -> list:
    images = []
    dir1 = "data/HAM10000_images_part_1/"
    dir2 = "data/HAM10000_images_part_2/"
    ext = ".jpg"
    for _id in images_ids:
        try:
            img = image.load_img(dir1+_id+ext, target_size=image_size)
            img = image.img_to_array(img)
            img = img.reshape((1, img.shape[0], img.shape[1], img.shape[2]))
            img = preprocess_input(img)
            images.append(img)
        except:
            try:
                img = image.load_img(dir2+_id+ext, target_size=image_size)
                img = image.img_to_array(img)
                img = img.reshape((1, img.shape[0], img.shape[1], img.shape[2]))
                img = preprocess_input(img)
                images.append(img)
            except:
                print(dir1 + _id)
                print(dir2 + _id)
                print("WOW")
            
            
    return np.vstack(images)

In [29]:
# Import the keras preprocessing method.
from tensorflow.keras.preprocessing import image
image_size = (224, 224)
# Load your images and preprocess them.

bkl_images = get_preprocessed_images(super_list[0],image_size)
nv_images = get_preprocessed_images(super_list[1],image_size)
df_images = get_preprocessed_images(super_list[2],image_size)
mel_images = get_preprocessed_images(super_list[3],image_size)
vasc_images = get_preprocessed_images(super_list[4],image_size)
bcc_images = get_preprocessed_images(super_list[5],image_size)
akiec_images = get_preprocessed_images(super_list[6],image_size)


data/HAM10000_images_part_1/ISIC_0032365
data/HAM10000_images_part_2/ISIC_0032365
WOW


In [ ]:
# Make a numpy array for each of the class labels (one hot encoded).
bkl_labels = np.tile([1, 0, 0, 0, 0, 0, 0], (bkl_images.shape[0], 1))
nv_labels = np.tile([0, 1, 0, 0, 0, 0, 0], (nv_images.shape[0], 1))
df_labels = np.tile([0, 0, 1, 0, 0, 0, 0], (df_images.shape[0], 1))
mel_labels = np.tile([0, 0, 0, 1, 0, 0, 0], (mel_images.shape[0], 1))
vasc_labels = np.tile([0, 0, 0, 0, 1, 0, 0], (vasc_images.shape[0], 1))
bcc_labels = np.tile([0, 0, 0, 0, 0, 1, 0], (bcc_images.shape[0], 1))
akiec_labels = np.tile([0, 0, 0, 0, 0, 0, 1], (akiec_images.shape[0], 1))

X = np.concatenate([bkl_images,
                    nv_images,
                    df_images,
                    mel_images,
                    vasc_images,
                    bcc_images,
                    akiec_images])

y = np.concatenate([bkl_labels,
                    nv_labels,
                    df_labels,
                    mel_labels,
                    vasc_labels,
                    bcc_labels,
                    akiec_labels])

print(X.shape, y.shape)


(10015, 224, 224, 3) (10015, 7)


In [ ]:
from sklearn.model_selection import train_test_split

X_train_val, X_test, y_train_val, y_test = train_test_split(
    X, 
    y,
    test_size=0.2, 
    random_state=42, 
    shuffle=True
)

X_train, X_val, y_train, y_val = train_test_split(
    X_train_val, 
    y_train_val,
    test_size=0.2, 
    random_state=42, 
    shuffle=True
)


In [ ]:
from tensorflow.keras.preprocessing.image import ImageDataGenerator

# Determine the number of generated samples you want per original sample.
datagen_batch_size = 16

# Make a datagenerator object using ImageDataGenerator.
train_datagen = ImageDataGenerator(rotation_range=60,
                                    horizontal_flip=True)

# Feed the generator your train data.
train_generator = train_datagen.flow(X_train, y_train, batch_size=datagen_batch_size)

# Make a datagenerator object using ImageDataGenerator.
validation_datagen = ImageDataGenerator(rotation_range=60,
                                        horizontal_flip=True)

# Feed the generator your validation data.
validation_generator = validation_datagen.flow(X_val, y_val, batch_size=datagen_batch_size)

In [ ]:
# Import your chosen model!
from tensorflow.keras.applications.mobilenet_v2 import MobileNetV2

# Make a model object. 
# Make sure you exclude the top part. set the input shape of the model to 224x224 pixels, with 3 color channels.
model = MobileNetV2(weights='imagenet', include_top=False, input_shape=(224,224,3))

# Freeze the imported layers so they cannot be retrained.
for layer in model.layers:
    layer.trainable = False
    
model.summary()

Model: "mobilenetv2_1.00_224"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_1 (InputLayer)            [(None, 224, 224, 3) 0                                            
__________________________________________________________________________________________________
Conv1 (Conv2D)                  (None, 112, 112, 32) 864         input_1[0][0]                    
__________________________________________________________________________________________________
bn_Conv1 (BatchNormalization)   (None, 112, 112, 32) 128         Conv1[0][0]                      
__________________________________________________________________________________________________
Conv1_relu (ReLU)               (None, 112, 112, 32) 0           bn_Conv1[0][0]                   
_______________________________________________________________________________

In [ ]:
from tensorflow.keras import Sequential
from tensorflow.keras.layers import Dense
from tensorflow.keras.layers import Flatten
from tensorflow.keras.layers import Dropout

new_model = Sequential()
new_model.add(model)
new_model.add(Flatten())
new_model.add(Dense(64, activation='relu'))
new_model.add(Dropout(0.5))
new_model.add(Dense(7, activation='sigmoid'))

# Summarize.
new_model.summary()

Model: "sequential_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
mobilenetv2_1.00_224 (Functi (None, 7, 7, 1280)        2257984   
_________________________________________________________________
flatten_1 (Flatten)          (None, 62720)             0         
_________________________________________________________________
dense_2 (Dense)              (None, 64)                4014144   
_________________________________________________________________
dropout_1 (Dropout)          (None, 64)                0         
_________________________________________________________________
dense_3 (Dense)              (None, 7)                 455       
Total params: 6,272,583
Trainable params: 4,014,599
Non-trainable params: 2,257,984
_________________________________________________________________


Physical devices cannot be modified after being initialized


In [ ]:


# Compile and fit the model. Use the Adam optimizer and crossentropical loss. 
# Use the validation data argument during fitting to include your validation data.
new_model.compile(optimizer='adam',
                  loss='binary_crossentropy',
                  metrics=['accuracy'])
history = new_model.fit(train_generator,
                        epochs=10, 
                        batch_size=8,
                        validation_data=validation_generator
                       )

In [ ]:
import tensorflow

def plot_history(history : tensorflow.python.keras.callbacks.History):
    """ This helper function takes the tensorflow.python.keras.callbacks.History
    that is output from your `fit` method to plot the loss and accuracy of
    the training and validation set.
    """
    fig, axs = plt.subplots(1,2, figsize=(12,6))
    axs[0].plot(history.history['accuracy'], label='training set')
    axs[0].plot(history.history['val_accuracy'], label = 'validation set')
    axs[0].set(xlabel = 'Epoch', ylabel='Accuracy', ylim=[0, 1])

    axs[1].plot(history.history['loss'], label='training set')
    axs[1].plot(history.history['val_loss'], label = 'validation set')
    axs[1].set(xlabel = 'Epoch', ylabel='Loss', ylim=[0, 10])
    
    axs[0].legend(loc='lower right')
    axs[1].legend(loc='lower right')
    
plot_history(history)